In [0]:
%sql
drop table vision_dev.vision.stg_IB_industry_country_spare

In [0]:
%sql
CREATE TABLE vision_dev.vision.stg_IB_industry_country_spare (
  order_date STRING,
  dealer_id DOUBLE,
  country STRING,
  part_id STRING,
  order_qty DOUBLE,
  type STRING,
  Material_Description STRING,
  industry STRING,
  ABC_FMS STRING,
  run_date STRING
);

In [0]:
%sql
select * from vision_dev.vision.stg_IB_industry_country_spare

order_date,dealer_id,country,part_id,order_qty,type,Material_Description,industry,ABC_FMS,run_date


In [0]:
# Read CSV into pandas
import pandas as pd
#df_pandas = pd.read_csv("/Workspace/Users/thejaswini.p@tvsmotor.com/IB_Forecast_June_Country.csv", encoding='ISO-8859-1')
df_pandas = pd.read_csv("//Workspace/Users/thejaswini.p@tvsmotor.com/IB_Forecast_Dec_Country_2025.csv", encoding='ISO-8859-1')
df_pandas['ABC_FMS'] = df_pandas['ABC_FMS'].fillna('A_F')
# Convert to Spark DataFrame
df_spark = spark.createDataFrame(df_pandas)

# Cast columns to required schema
df_casted = df_spark.selectExpr(
    "CAST(order_date AS STRING) AS order_date",
    "CAST(dealer_id AS DOUBLE) AS dealer_id",
    "CAST(country AS STRING) AS country",
    "CAST(part_id AS STRING) AS part_id",
    "CAST(order_qty AS DOUBLE) AS order_qty",
    "CAST(type AS STRING) AS type",
    "CAST(`Material Description` AS STRING) AS Material_Description",
    "CAST(industry AS STRING) AS industry",
    "CAST(ABC_FMS AS STRING) AS ABC_FMS",
    "CAST(run_date AS STRING) AS run_date"
)

# Create/replace temp view with the desired schema
df_casted.createOrReplaceTempView("temp_IB_Forecast_prof_Country")


In [0]:
df_pandas.head(5)

,country,part_id,order_date,order_qty,run_date,type,ABC_FMS,dealer_id,Material Description,industry
0,Nepal,1110020,1/12/2025,6827,1/12/2025,Forecast,A_F,70404,SHOE SET BRAKE 130 DIA/28MM WIDTH,2W
1,Nepal,1110020,1/1/2026,4482,1/12/2025,Forecast,A_F,70404,SHOE SET BRAKE 130 DIA/28MM WIDTH,2W
2,Nepal,1110020,1/2/2026,5405,1/12/2025,Forecast,A_F,70404,SHOE SET BRAKE 130 DIA/28MM WIDTH,2W
3,Nepal,1110020,1/3/2026,4640,1/12/2025,Forecast,A_F,70404,SHOE SET BRAKE 130 DIA/28MM WIDTH,2W
4,Nepal,1110020,1/4/2026,4555,1/12/2025,Forecast,A_F,70404,SHOE SET BRAKE 130 DIA/28MM WIDTH,2W


In [0]:
%sql
select * from temp_IB_Forecast_prof_Country limit 10

order_date,dealer_id,country,part_id,order_qty,type,Material_Description,industry,ABC_FMS,run_date
1/12/2025,70404.0,Nepal,1110020,6827.0,Forecast,SHOE SET BRAKE 130 DIA/28MM WIDTH,2W,A_F,1/12/2025
1/1/2026,70404.0,Nepal,1110020,4482.0,Forecast,SHOE SET BRAKE 130 DIA/28MM WIDTH,2W,A_F,1/12/2025
1/2/2026,70404.0,Nepal,1110020,5405.0,Forecast,SHOE SET BRAKE 130 DIA/28MM WIDTH,2W,A_F,1/12/2025
1/3/2026,70404.0,Nepal,1110020,4640.0,Forecast,SHOE SET BRAKE 130 DIA/28MM WIDTH,2W,A_F,1/12/2025
1/4/2026,70404.0,Nepal,1110020,4555.0,Forecast,SHOE SET BRAKE 130 DIA/28MM WIDTH,2W,A_F,1/12/2025
1/5/2026,70404.0,Nepal,1110020,5024.0,Forecast,SHOE SET BRAKE 130 DIA/28MM WIDTH,2W,A_F,1/12/2025
1/12/2025,70157.0,Sri Lanka,1110020,2570.0,Forecast,SHOE SET BRAKE 130 DIA/28MM WIDTH,2W,A_F,1/12/2025
1/1/2026,70157.0,Sri Lanka,1110020,764.0,Forecast,SHOE SET BRAKE 130 DIA/28MM WIDTH,2W,A_F,1/12/2025
1/2/2026,70157.0,Sri Lanka,1110020,731.0,Forecast,SHOE SET BRAKE 130 DIA/28MM WIDTH,2W,A_F,1/12/2025
1/3/2026,70157.0,Sri Lanka,1110020,1493.0,Forecast,SHOE SET BRAKE 130 DIA/28MM WIDTH,2W,A_F,1/12/2025


In [0]:
%sql
CREATE OR REPLACE TABLE vision_dev.vision.IB_industry_country_spare_temp AS
SELECT * 
FROM vision_dev.vision.IB_industry_country_spare target
WHERE NOT EXISTS (
  SELECT 1
  FROM temp_IB_Forecast_prof_Country source
  WHERE
    target.order_date = source.order_date AND
    target.dealer_id = CAST(source.dealer_id AS BIGINT) AND
    target.country = source.country AND
    target.part_id = source.part_id AND
    target.type = source.type
);


num_affected_rows,num_inserted_rows


In [0]:
%sql
INSERT INTO vision_dev.vision.IB_industry_country_spare_temp
SELECT
  to_date(order_date, 'yy/MM/dd'),
  CAST(dealer_id AS BIGINT),
  country,
  part_id,
  CAST(order_qty AS BIGINT),
  Material_Description,
  industry,
  ABC_FMS,
  --run_date,
  to_date(run_date, 'yy/MM/dd'),
  type
FROM temp_IB_Forecast_prof_Country;


num_affected_rows,num_inserted_rows
4116,4116


In [0]:
%sql
select count(*) from vision_dev.vision.IB_industry_country_spare_temp

count(1)
35325583


In [0]:
%sql
drop table vision_dev.vision.IB_industry_country_spare

In [0]:
%sql
CREATE TABLE vision_dev.vision.IB_industry_country_spare AS
SELECT *
FROM vision_dev.vision.IB_industry_country_spare_temp;

num_affected_rows,num_inserted_rows
